In [ ]:
rm(list=ls())
libraries = c("dplyr","magrittr","tidyr","reshape2","ggplot2","ggpubr","extrafont","openxlsx","readxl","RColorBrewer","MetBrewer","zoo"
              ,"writexl","gridExtra","MASS","readr","stats","boot","pracma","stringr","mixdist","corpcor","rstan","bayesplot","graphics","crone","bignum")
for(x in libraries) { library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE) }

theme_set(theme_bw())
setwd(".../MPX_depletion_susceptibles")

In [ ]:
setwd("C:/Users/USER/Documents/GitHub/mokeypox_depletion_of_susceptibles")

In [ ]:
alpha = 0.1; kappa=0.77;#alpha = 0.1; kappa=0.88; # alpha = 0.14; kappa=0.77# alpha = 0.06; kappa=0.77; # alpha = 0.1; kappa=0.66; # alpha = 0.1; kappa=0.88; 
theta=(14/365)*((alpha/kappa)^(1/alpha));

dweibull_lt = function (x,shape,scale,u){
  if(min(x)<u){
      return(0)
      #stop('The value is less than left truncated point')
  }
  (dweibull(x,shape,scale)/(1-pweibull(u,shape,scale)))*heaviside(x,u)
}

pweibull_lt = function(q,shape,scale,u){
  if(min(q)<u){
      return(0)
      #stop('The quantile is less than left truncated point')
  }
  (pweibull(q,shape,scale)-pweibull(u,shape,scale))/
    (1-pweibull(u,shape,scale))
}

aa <- rep(0,1000*365)
width <- 1/365
for(t in 1:(1000*365)){
    x <- t*width
    aa[t] <- x*dweibull_lt(x,alpha,theta,14/365)
}
mu1 <- sum(aa)*width
mu1

In [ ]:
upperl = 1001
S = rep(0,(upperl-1)/width)
    for(t in 1:((upperl-1)/width)){
        x <- width*t 
        S[t] = width*dweibull_lt(x,alpha,theta,14/365)
    }
wei_sum = sum(S)
wei_sum

In [ ]:
pweibull_lt = function(q,shape,scale,u){
  if(min(q)<u){
    return(0)
    #stop('The quantile is less than left truncated point')
  }
  (pweibull(q,shape,scale)-pweibull(u,shape,scale))/
    (1-pweibull(u,shape,scale))
}
alpha = 0.1; kappa=0.77; 
theta=(14/365)*((alpha/kappa)^(1/alpha));

1000000*(1-pweibull_lt(10,alpha,theta,7/365))

In [ ]:
### set weibull parameters
alpha = 0.1; kappa=0.77; 
theta=(14/365)*((alpha/kappa)^(1/alpha));

dweibull_lt = function (x,shape,scale,u){
  if(min(x)<u){
      return(0)
      #stop('The value is less than left truncated point')
  }
  (dweibull(x,shape,scale)/(1-pweibull(u,shape,scale)))*heaviside(x,u)
}

aa <- rep(0,1000*365)
width <- 1/365
for(t in 1:(1000*365)){
    x <- t*width
    aa[t] <- x*dweibull_lt(x,alpha,theta,14/365)
}
mu1 <- sum(aa)*width ### weibull mean

width <- 1/365
N <- 1000000
upperl = 1001
S = rep(0,(upperl-1)/width)
    for(t in 1:((upperl-1)/width)){
        x <- width*t 
        S[t] = width*dweibull_lt(x,alpha,theta,14/365)
    }
wei_sum = sum(S)


### simulation code
Simulation_fn = function(n){
    upperl =1001
    fx = rep(0,(upperl-1)/width)
    sx = rep(0,(upperl-1)/width)
    meanx = rep(0,(upperl-1)/width)
    for(t in 1:((upperl-1)/width)){
        x <- width*t 
        fx[t] = width*x*max(0,x-1)*dweibull_lt(x,alpha,theta,14/365)*exp(-(x*n)/(mu1*N))
        sx[t] = width*dweibull_lt(x,alpha,theta,14/365)*exp(-(x*n)/(mu1*N))
        meanx[t] = width*x*dweibull_lt(x,alpha,theta,14/365)*exp(-(x*n)/(mu1*N))
    }
    
        meanx1 = rep(0,9/width)
    for(t in 1:(9/width)){
        x <- width*t 
        meanx1[t] = width*x*dweibull_lt(x,alpha,theta,14/365)*exp(-(x*n)/(mu1*N))
    }
    
    excess= sum(meanx1[1:(9/width)])/sum(meanx[1:((upperl-1)/width)])   ### mean excess degree of susceptibles
    df = c(excess) 
    return(df)
}

df <- mapply(Simulation_fn,seq(0,100*30,100))
1-df

In [ ]:
result_df$Infections[1:30]/df_SAR_Reff$Infections[1:30]

In [ ]:
result_df$Infections[1:30]/df_SAR_Reff$Infections[1:30]

In [ ]:
source("src/R/inference.R")

In [ ]:
result_df <- cbind(result_df,seq(0,100*800,100))
colnames(result_df) <- c("SAR","Infections","Reff_1","Reff_2","excess","n")
write.csv(result_df, "output/csv/inference/SAR_cip_Reff_excess.csv")

In [ ]:
df_SAR_Reff <- read.csv("output/csv/inference/SAR_m_Reff.csv")
head(df_SAR_Reff)

In [ ]:
inte_df3 <- rbind(c(0,df_SAR_Reff$Infections[1:500]),c(0,as.numeric(df_SAR_Reff$SAR[1:500]))) %>% t() %>% as.data.frame()
colnames(inte_df3) <- c("m","SAR")
inte_df3

In [ ]:
inte_df2 <- rbind(df_SAR_Reff$Infections[1:400],df_SAR_Reff$SAR[1:400]) %>% t() %>% as.data.frame()
colnames(inte_df2) <- c("m","SAR")
inte_df2

In [ ]:
inte_df <- c(df_SAR_Reff$Reff_1[1:100],df_SAR_Reff$Reff_2[1:100],3*df_SAR_Reff$Reff_1[1:100])
mm <- c(df_SAR_Reff$Infections[1:100],df_SAR_Reff$Infections[1:100],df_SAR_Reff$Infections[1:100])
inte_df1 <- rbind(mm,inte_df,c(rep("SAR=0.1",100),rep("SAR=0.2",100),rep("SAR=0.3",100))) %>% t() %>% as.data.frame()
colnames(inte_df1) <- c("m","Reff","label")
inte_df1

In [ ]:
df_SAR_Reff11 <- read.csv("output/csv/inference/SAR_m_Reff_sa11.csv")
df_SAR_Reff12 <- read.csv("output/csv/inference/SAR_m_Reff_sa12.csv")
df_SAR_Reff <- read.csv("output/csv/inference/SAR_m_Reff.csv")
df_SAR_Reff21 <- read.csv("output/csv/inference/SAR_m_Reff_sa21.csv")
df_SAR_Reff22 <- read.csv("output/csv/inference/SAR_m_Reff_sa22.csv")


inte_df11 <- rbind(df_SAR_Reff11$Infections[1:400],df_SAR_Reff11$SAR[1:400]) %>% t() %>% as.data.frame()
inte_df12 <- rbind(df_SAR_Reff12$Infections[1:400],df_SAR_Reff12$SAR[1:400]) %>% t() %>% as.data.frame()
inte_df <- rbind(df_SAR_Reff$Infections[1:400],df_SAR_Reff$SAR[1:400]) %>% t() %>% as.data.frame()
inte_df21 <- rbind(df_SAR_Reff21$Infections[1:400],df_SAR_Reff21$SAR[1:400]) %>% t() %>% as.data.frame()
inte_df22 <- rbind(df_SAR_Reff22$Infections[1:400],df_SAR_Reff22$SAR[1:400]) %>% t() %>% as.data.frame()

inte_df_x <- rbind(inte_df,inte_df11,inte_df12,inte_df21,inte_df22)
label_1 <- c(rep("baseline (1%tile:16; κ=0.77)",400),rep("1%tile: 10; κ=0.77",400),rep("1%tile: 30; κ=0.77",400),rep("1%tile: 16; κ=0.66",400),rep("1%tile: 16; κ=0.88",400))
inte_df_x <- data.frame(inte_df_x, label_1=factor(label_1, levels=c("baseline (1%tile:16; κ=0.77)","1%tile: 10; κ=0.77", "1%tile: 30; κ=0.77", "1%tile: 16; κ=0.66", "1%tile: 16; κ=0.88")))
colnames(inte_df_x) <- c("m","SAR","label_1")


range_l_country <- as.numeric(inte_df2$m[14])
range_u_country <- as.numeric(inte_df2$m[43])
range_l_us_state <- as.numeric(inte_df2$m[14])
range_u_us_state <- as.numeric(inte_df2$m[73])

options(repr.plot.width=18,repr.plot.height=8)
options(scipen=1)
pooled_sar_l_country <- round(as.numeric(inte_df2$SAR[14])*10000)/10000
pooled_sar_u_country <- round(as.numeric(inte_df2$SAR[43])*10000)/10000
pooled_sar_l_us_state <- round(as.numeric(inte_df2$SAR[14])*10000)/10000
pooled_sar_u_us_state <- round(as.numeric(inte_df2$SAR[73])*10000)/10000

options(repr.plot.width=18,repr.plot.height=10)
    ggplot() +
    geom_segment(aes(x=range_l_us_state*100,xend=range_l_us_state*100,y=0,yend=Inf), linetype=1, colour=met.brewer("Pissaro",n=7)[3], size=1) +
    geom_segment(aes(x=range_u_us_state*100,xend=range_u_us_state*100,y=0,yend=Inf), linetype=1, colour=met.brewer("Pissaro",n=7)[3], size=1) +
    geom_segment(aes(x=range_l_country*100,xend=range_l_country*100,y=0,yend=Inf), linetype=4, colour=met.brewer("Pissaro",n=7)[2], size=1.2) + # country
    geom_segment(aes(x=range_u_country*100,xend=range_u_country*100,y=0,yend=Inf), linetype=4, colour=met.brewer("Pissaro",n=7)[2], size=1.2) +
    geom_rect(aes(xmin = range_l_country*100, xmax = range_u_country*100, ymin = 0, ymax = Inf, fill = 'Countries'), alpha = 0.3) + # rect country
    geom_rect(aes(xmin = range_l_us_state*100, xmax = range_u_us_state*100, ymin = 0, ymax = Inf, fill = 'US states'),alpha=0.2) + # rect us states
    geom_line(inte_df_x,mapping=aes(x=as.numeric(m)*100, y=as.numeric(SAR), colour=factor(label_1), linetype=label_1), size=1.5, alpha=2) + 
    theme_bw(base_size = 22) + theme(legend.text=element_text(size=20))  + theme(axis.text = element_text(size = 20)) +
    scale_fill_manual(breaks=c('Countries','US states'),values=c('Countries' = met.brewer("Pissaro",n=7)[2],'US states' = met.brewer("Pissaro",n=7)[3]),label=c("Countries","US states")) +
    scale_color_manual(name = "NAME", values=c(met.brewer("Ingres",n=8)[1],met.brewer("Homer2",n=6)[1],met.brewer("Homer2",n=6)[2] ,met.brewer("OKeeffe1",n=11)[9], met.brewer("OKeeffe1",n=11)[11]))  +
    scale_linetype_manual(name = "NAME", values = c(1,2,2,3,3)) +
    scale_x_continuous(breaks = seq(0,0.8,0.2), limits=c(0,0.8)) + scale_y_continuous(breaks=c(seq(0,0.6,0.1)),limits=c(0,0.6)) +
    theme(legend.position="right", legend.title = element_blank(), legend.background = element_blank(), legend.key.size = unit(1.8, 'cm')) +
    labs(x="Cumulative cases per MSM population\
at a peak of an epidemic [%]\
 ", y = "Secondary attack risk\
 ")
ggsave(file = "output/fig/sensitivity_weibull.jpeg", width = 18, height = 10)

In [ ]:
range_l_country <- as.numeric(inte_df2$m[14])#0.0013
range_u_country <- as.numeric(inte_df2$m[43])#0.00391
range_l_us_state <- as.numeric(inte_df2$m[15])#0.00137
range_u_us_state <- as.numeric(inte_df2$m[74])#0.00655

options(repr.plot.width=18,repr.plot.height=8)
options(scipen=1)
pooled_sar_l_country <- round(as.numeric(inte_df2$SAR[14])*10000)/10000
pooled_sar_u_country <- round(as.numeric(inte_df2$SAR[43])*10000)/10000
pooled_sar_l_us_state <- round(as.numeric(inte_df2$SAR[15])*10000)/10000
pooled_sar_u_us_state <- round(as.numeric(inte_df2$SAR[74])*10000)/10000

SAR_Inf_plot <- ggplot()+
    geom_segment(aes(x=range_l_us_state*100,xend=range_l_us_state*100,y=0,yend=pooled_sar_l_us_state), linetype=1, colour=met.brewer("Pissaro",n=7)[3], size=1) + # us states
    geom_segment(aes(x=0,xend=range_l_us_state*100,y=pooled_sar_l_us_state,yend=pooled_sar_l_us_state), linetype=1, colour=met.brewer("Pissaro",n=7)[3],size=1) +
    geom_segment(aes(x=range_u_us_state*100,xend=range_u_us_state*100,y=0,yend=pooled_sar_u_us_state), linetype=1, colour=met.brewer("Pissaro",n=7)[3], size=1) +
    geom_segment(aes(x=0,xend=range_u_us_state*100,y=pooled_sar_u_us_state,yend=pooled_sar_u_us_state), linetype=1, colour=met.brewer("Pissaro",n=7)[3], size=1) +
    geom_segment(aes(x=range_l_country*100,xend=range_l_country*100,y=0,yend=pooled_sar_l_country), linetype=4, colour=met.brewer("Pissaro",n=7)[2], size=1.2) + # country
    geom_segment(aes(x=0,xend=range_l_country*100,y=pooled_sar_l_country,yend=pooled_sar_l_country), linetype=4, colour=met.brewer("Pissaro",n=7)[2], size=1) +
    geom_segment(aes(x=range_u_country*100,xend=range_u_country*100,y=0,yend=pooled_sar_u_country), linetype=4, colour=met.brewer("Pissaro",n=7)[2], size=1.2) +
    geom_segment(aes(x=0,xend=range_u_country*100,y=pooled_sar_u_country,yend=pooled_sar_u_country), linetype=4, colour=met.brewer("Pissaro",n=7)[2], size=1.2) +
    geom_rect(aes(xmin = 0, xmax = range_u_country*100, ymin = pooled_sar_l_country, ymax = pooled_sar_u_country, fill = 'Countries'), alpha = 0.3, inherit.aes = FALSE) + # rect country
    geom_rect(aes(xmin = range_l_country*100, xmax = range_u_country*100, ymin = 0, ymax = pooled_sar_l_country, fill = 'Countries'), alpha = 0.3, inherit.aes = FALSE) +
    geom_rect(aes(xmin = 0, xmax = range_u_us_state*100, ymin = pooled_sar_l_us_state, ymax = pooled_sar_u_us_state, fill = 'US states'),alpha=0.2,inherit.aes = FALSE) + # rect us states
    geom_rect(aes(xmin = range_l_us_state*100, xmax = range_u_us_state*100, ymin = 0, ymax = pooled_sar_l_us_state, fill = 'US states'),alpha=0.2, inherit.aes = FALSE) +
    geom_line(inte_df2,mapping=aes(x=as.numeric(m)*100, y=as.numeric(SAR)), colour=met.brewer("Greek",n=3)[2], size=1.5, alpha=6) + theme_bw(base_size = 22) + 
    theme(legend.text=element_text(size=20)) + theme(axis.text = element_text(size = 20)) +
    #scale_linetype_manual(breaks=c('Countries','US states'), values = c('Countries'=4, 'US states'=1)) +
    scale_fill_manual(breaks=c('Countries','US states'),values=c('Countries' = met.brewer("Pissaro",n=7)[2],'US states' = met.brewer("Pissaro",n=7)[3]),label=c("Countries","US states")) +
    theme(legend.position=c(0.75,0.85), legend.title = element_blank(), legend.background = element_blank(), legend.key.size = unit(1.5, 'cm')) +
    labs(x="Cumulative cases per MSM population\
at a peak of an epidemic [%]\
 ", y = "Secondary attack risk\
 ") + scale_x_continuous(breaks = seq(0,0.8,0.2), limits=c(0,0.8)) + scale_y_continuous(breaks=c(seq(0,0.4,0.1)),limits=c(0,0.4)) 

Reff_Inf_plot <-  inte_df1 %>% ggplot() +
    geom_hline(yintercept=1, linetype="dashed", color = "#bdbdbd", size =1.8,alpha=2) +
    geom_line(aes(x=as.numeric(m)*100, y=as.numeric(Reff), group=label, colour=label), size=1.5, alpha=2) +
    theme_bw(base_size = 22)+theme(legend.text=element_text(size=20))  + theme(axis.text = element_text(size = 20)) + theme(legend.position=c(0.85,0.85), legend.title = element_blank(), legend.background = element_blank(), legend.key.size = unit(1.5, 'cm')) + guides(fill=guide_legend(""))  + 
    scale_color_manual(values=c(met.brewer("Troy",n=15)[3],met.brewer("Troy",n=15)[8],met.brewer("Troy",n=15)[12])) +
    labs(x="Cumulative cases per MSM population\
at a peak of an epidemic [%]\
 ", y = "Effective reprodunction number\
 ") #+ scale_x_continuous(breaks = seq(0,0.009,0.002), limits=c(0,0.009)) +
    #scale_y_continuous(breaks=c(0,2,4),limits=c(0,4))

ggarrange(Reff_Inf_plot, SAR_Inf_plot, nrow=1,ncol=2,font.label=list(size=25),labels=c("",""))
ggsave(file = "output/fig/SAR_Reffvsprop.jpeg", width = 18, height = 8)

In [ ]:
inte_df3 <- rbind(df_SAR_Reff$Infections[1:127],df_SAR_Reff$SAR[1:127]) %>% t() %>% as.data.frame()
colnames(inte_df3) <- c("m","SAR")

In [ ]:
source("src/R/finalsize.R")

In [ ]:
options(repr.plot.width=10,repr.plot.height=8)
    ggplot() +
    geom_line(inte_df3,mapping=aes(y=as.numeric(m)*100, x=SAR, colour="Peak"), size=1.5, alpha=2) + 
    geom_line(result_df_final,mapping=aes(y=case*100, x=SAR, colour="Final size",), size=1.5, alpha=2) +
    theme_bw(base_size = 22)+theme(legend.text=element_text(size=20))  + theme(axis.text = element_text(size = 20)) +  scale_color_manual(breaks=c('Peak','Final size'),values=c('Peak'=met.brewer("Monet",n=9)[2],'Final size'=met.brewer("Monet",n=9)[8]),label=c("Peak","Final size"))  +
    theme(legend.position=c(0.2,0.85),legend.background = element_blank(), legend.title = element_blank(), legend.key.size = unit(1.5, 'cm')) +
    labs(y="Cumulative cases per MSM population\
at a peak of an epidemic [%]\
 ", x = "Secondary attack risk\
 ") + scale_y_continuous(breaks = seq(0,4,1), limits=c(0,4)) + 
scale_x_continuous(breaks=c(seq(0,0.4,0.1)),limits=c(0,0.4)) 
ggsave(file = "output/fig/finalsize_peak_vsSAR.jpeg", width = 10, height = 8)